In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Load election data
data_dir = "../data/"
election_data = pd.read_csv(data_dir + "G20192-Columnes-ME-EN.csv", sep=';')

In [ ]:
# Load GeoJSON file
map_catalonia='county_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)
geo_data

In [ ]:
election_data

In [ ]:
geo_data.rename(columns={'nomcomar': 'County_name'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='County_name')

In [ ]:
election_data

In [ ]:
participation_by_county = election_data.iloc[:, 11:21].groupby(by=election_data["County_name"]).sum()
participation_by_county.reset_index()
participation_by_county

In [ ]:
participation_by_county['Final part/Census'] = participation_by_county['Part_20:00'] / participation_by_county['Electoral_census']
participation_by_county

In [ ]:
merged_data = geo_data_sorted.merge(participation_by_county, on='County_name')
merged_data

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Final part/Census',  # Change to the column you want to visualize
    hover_name='County_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, width=800)
fig.show()